# Fine Tuning Open AI Models

Dis notebook base on di current guidance wey dem provide for di [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) documentation from Open AI.

Fine-tuning dey improve di performance of foundation models for your application by retraining am with extra data and context wey relate to dat particular use case or scenario. Make you sabi say prompt engineering techniques like _few shot learning_ and _retrieval augmented generation_ dey allow you to improve di default prompt with relevant data to make e better. But, dis kain approach get limit because of di max token window size of di foundation model wey you dey target.

With fine-tuning, we dey effectively retrain di model itself with di data wey we need (dis one go allow us use plenty examples pass wetin fit for di max token window) - and we go deploy a _custom_ version of di model wey no need make you provide examples when you dey do inference. Dis one no just improve how we dey design our prompt (because we get more freedom to use di token window for other tins) but e fit also reduce our cost (because e go reduce di number of tokens we need to send to di model when we dey do inference).

Fine tuning get 4 steps:
1. Prepare di training data and upload am.
1. Run di training job to get fine-tuned model.
1. Check di fine-tuned model and improve am for quality.
1. Deploy di fine-tuned model for inference when you satisfy.

Make you sabi say no be all foundation models fit support fine-tuning - [check OpenAI documentation](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) for di latest info. You fit also fine-tune model wey dem don fine-tune before. For dis tutorial, we go use `gpt-35-turbo` as our target foundation model for fine-tuning. 

---


### Step 1.1: Prepare Your Dataset

Make we build chatbot wey go help you sabi the periodic table of elements by answering questions about one element wit limerick. For _this_ simple tutorial, we go just create dataset to train the model wit small sample examples of responses wey show how the data suppose be. For real-world use case, you go need create dataset wit plenty examples. You fit also fit use open dataset (for your application domain) if e dey, and reformat am to use for fine-tuning.

Since we dey focus on `gpt-35-turbo` and we dey find single-turn response (chat completion) we fit create examples using [this suggested format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) wey reflect OpenAI chat completion requirements. If you expect multi-turn conversational content, you go use the [multi-turn example format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) wey get `weight` parameter to show which messages suppose dey used (or no) for fine-tuning process.

We go use the simpler single-turn format for our tutorial here. The data dey for [jsonl format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) wit 1 record per line, each one represent as JSON-formatted object. The snippet below show 2 records as sample - see [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) for full sample set (10 examples) we go use for our fine-training tutorial. **Note:** Each record _must_ dey defined for single line (no split across lines like normal formatted JSON file)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

For real-world use case you go need bigger examples set for better results - the tradeoff na between quality of responses and the time/costs for fine-tuning. We dey use small set so we fit finish fine-tuning quick to show the process. See [this OpenAI Cookbook example](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) for more complex fine-tuning tutorial.


---

### Step 1.2 Upload Your Dataset

Upload di data using di Files API [as dem tok for here](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Make you sabi say to fit run dis code, you gats don do dis steps first:
 - Install di `openai` Python package (make sure say you use version wey big pass or equal to 0.28.0 for di latest features)
 - Set di `OPENAI_API_KEY` environment variable to your OpenAI API key
To sabi more, check di [Setup guide](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) wey dem provide for di course.

Now, run di code to create file wey you go upload from your local JSONL file.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Step 2.1: Create di Fine-tuning job wit di SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Step 2.2: Check di Status of di job

Here na some tins wey you fit do wit di `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - List di last n fine-tuning jobs
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Get details of one fine-tuning job
- `client.fine_tuning.jobs.cancel(<job_id>)` - Cancel one fine-tuning job
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - List up to n events from di job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Di first step for di process na _validating di training file_ to make sure say data dey for correct format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Step 2.3: Track events to monitor progress


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Step 2.4: See how e dey for OpenAI Dashboard


You fit also check di status by going OpenAI website and explore di _Fine-tuning_ section for di platform. Dis one go show you di status of di current job, and e go also allow you track di history of previous job execution runs. For dis screenshot, you fit see say di previous execution fail, and di second run succeed. For context, dis one happen when di first run use JSON file wey get wrong format records - after dem fix am, di second run complete well well and e make di model ready for use.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba.pcm.png)


You fit also see di status messages and metrics by scrolling down more for di visual dashboard as e show:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b.pcm.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a65229.pcm.png)|


---

### Step 3.1: Collect ID & Test Fine-Tuned Model for Code


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Step 3.2: Load & Test Fine-Tuned Model for Playground

You fit test di fine-tuned model for two ways now. First, you fit waka go Playground and use di Models drop-down to select your new fine-tuned model from di options wey dem list. Di other way na to use di "Playground" option wey dey for di Fine-tuning panel (see screenshot above) wey go launch dis _comparitive_ view wey dey show di foundation and fine-tuned model versions side-by-side for quick evaluation.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016.pcm.png)

Just fill di system context wey you use for your training data and put your test question. You go notice say both sides go update with di same context and question. Run di comparison and you go see di difference for outputs between dem. _Note how di fine-tuned model dey render di response for di format wey you provide for your examples while di foundation model just dey follow di system prompt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350.pcm.png)

You go notice say di comparison still dey provide di token counts for each model, and di time wey e take for di inference. **Dis specific example na simple one wey dem use to show di process but e no really reflect real world dataset or scenario**. You fit notice say both samples show di same number of tokens (system context and user prompt na di same) with di fine-tuned model dey take more time for inference (custom model).

For real-world scenarios, you no go dey use toy example like dis, but you go dey fine-tune against real data (e.g., product catalog for customer service) wey di quality of response go dey more clear. For _that_ context, to get response quality wey equal with di foundation model, e go need more custom prompt engineering wey go increase token usage and fit increase di processing time for inference. _To try am, check di fine-tuning examples for OpenAI Cookbook to start._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:
Dis document na AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator) wey translate am. Even though we dey try make am correct, abeg sabi say automated translation fit get some mistakes or no too correct. The original document wey e dey for im own language na the correct one. If na serious matter, e better make person wey sabi human translation do am. We no go take responsibility if person no understand well or if dem use dis translation do wrong thing.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
